In [1]:
import os
from osgeo import gdal
import pyproj
import requests
import urllib
import sys

# Define the approximate bounding box for the Uinta Basin
# These coordinates roughly encompass the basin
UINTA_BBOX = {
    'xmin': -110.25,  # Western boundary
    'xmax': -109.0,   # Eastern boundary
    'ymin': 39.75,    # Southern boundary
    'ymax': 40.75     # Northern boundary
}

# Using the original DATASETS_DICT
DATASETS_DICT = {
    'DEM_1m': 'Digital Elevation Model (DEM) 1 meter',
    'NED_1-3as': 'National Elevation Dataset (NED) 1/3 arc-second',
    'NED_1as': 'National Elevation Dataset (NED) 1 arc-second'
}

BASEURL = 'https://tnmaccess.nationalmap.gov/api/v1/products?'
MAXITEMS = 500

def get_uinta_elevation_data(dataset='NED_1-3as', output_folder='uinta_dem_data'):
    """
    Retrieve elevation data for the Uinta Basin.

    Args:
        dataset (str): Type of elevation data to retrieve. Default is 1/3 arc-second NED
        output_folder (str): Folder to save the downloaded data
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Construct the query URL
    bbox_str = f"{UINTA_BBOX['xmin']},{UINTA_BBOX['ymin']},{UINTA_BBOX['xmax']},{UINTA_BBOX['ymax']}"

    # Verify dataset is valid
    if dataset not in DATASETS_DICT:
        raise ValueError(f"Dataset {dataset} not found. Available datasets: {list(DATASETS_DICT.keys())}")

    # Construct query parameters
    params = {
        'bbox': bbox_str,
        'datasets': DATASETS_DICT[dataset],
        'max': MAXITEMS,
        'prodFormats': ''
    }

    # Make the API request
    try:
        response = requests.get(BASEURL, params=params)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error making API request: {e}")
        return None

    if 'items' not in data or not data['items']:
        print("No data found for the specified region")
        return None

    # Get download URLs
    download_urls = [item['downloadURL'] for item in data['items']]

    # Download the files
    downloaded_files = []
    for url in download_urls:
        filename = url.split('/')[-1]
        output_path = os.path.join(output_folder, filename)

        print(f"Downloading {filename}...")
        try:
            urllib.request.urlretrieve(url, output_path)
            downloaded_files.append(output_path)
        except Exception as e:
            print(f"Error downloading {filename}: {e}")
            continue

    return downloaded_files

def merge_uinta_dems(input_files, output_file='uinta_merged.tif'):
    """
    Merge downloaded DEM files into a single raster.

    Args:
        input_files (list): List of downloaded DEM files
        output_file (str): Name for the merged output file
    """
    # Set up the merge options
    merge_opts = gdal.WarpOptions(
        resampleAlg='cubic',
        multithread=True,
        format='GTiff',
        outputBounds=[UINTA_BBOX['xmin'], UINTA_BBOX['ymin'],
                     UINTA_BBOX['xmax'], UINTA_BBOX['ymax']]
    )

    # Merge the files
    try:
        gdal.Warp(output_file, input_files, options=merge_opts)
        print(f"Successfully merged DEMs to {output_file}")
    except Exception as e:
        print(f"Error merging DEMs: {e}")

def extract_cross_section(dem_file, start_point, end_point, num_points=1000):
    """
    Extract elevation values along a cross-section line.

    Args:
        dem_file (str): Path to the merged DEM file
        start_point (tuple): (lon, lat) of start point
        end_point (tuple): (lon, lat) of end point
        num_points (int): Number of points to sample along the line

    Returns:
        tuple: (distances, elevations) arrays
    """
    import numpy as np

    # Open the DEM
    dem = gdal.Open(dem_file)
    if dem is None:
        raise ValueError("Could not open DEM file")

    # Get geotransform
    geotransform = dem.GetGeoTransform()
    band = dem.GetRasterBand(1)

    # Create points along the line
    lons = np.linspace(start_point[0], end_point[0], num_points)
    lats = np.linspace(start_point[1], end_point[1], num_points)

    # Get elevations
    elevations = []
    distances = []

    # Calculate total distance for reference
    total_dist = np.sqrt((end_point[0] - start_point[0])**2 +
                        (end_point[1] - start_point[1])**2) * 111.32  # approx km per degree

    for i in range(num_points):
        # Convert to pixel coordinates
        px = int((lons[i] - geotransform[0]) / geotransform[1])
        py = int((lats[i] - geotransform[3]) / geotransform[5])

        # Get elevation
        try:
            elevation = band.ReadAsArray(px, py, 1, 1)[0][0]
            elevations.append(elevation)

            # Calculate distance in km from start point
            dist = np.sqrt((lons[i] - start_point[0])**2 +
                         (lats[i] - start_point[1])**2) * 111.32
            distances.append(dist)
        except:
            continue

    return np.array(distances), np.array(elevations)

if __name__ == '__main__':
    # Example usage
    output_dir = 'uinta_dem_data'

    # Download the DEM data
    print("Downloading DEM data for Uinta Basin...")
    downloaded_files = get_uinta_elevation_data(dataset='NED_1-3as', output_folder=output_dir)

    if downloaded_files:
        # Merge the DEMs
        merged_dem = 'uinta_merged.tif'
        print("Merging DEM files...")
        merge_uinta_dems(downloaded_files, merged_dem)

        # Example cross-section (adjust these coordinates for your specific needs)
        # These coordinates cut across the basin from southwest to northeast
        start_point = (-110.1, 40.0)  # SW point
        end_point = (-109.2, 40.5)    # NE point

        print("Extracting cross-section...")
        distances, elevations = extract_cross_section(merged_dem, start_point, end_point)

        # Save cross-section data
        import numpy as np
        output_data = np.column_stack((distances, elevations))
        np.savetxt('uinta_cross_section.csv', output_data,
                  delimiter=',', header='distance_km,elevation_m', comments='')

        print("Process complete! Cross-section data saved to 'uinta_cross_section.csv'")

Merging DEM files...
Successfully merged DEMs to uinta_merged.tif
Extracting cross-section...
Process complete! Cross-section data saved to 'uinta_cross_section.csv'
